## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [ ]:
file_name="/home/francesco/data/data_000637.txt.1"
N_rows=np.random.randint(10000,1310720)
df=pd.read_csv(file_name, nrows=N_rows)
df

2. Find out the number of BX in a ORBIT (the value 'x').

In [ ]:
df.groupby('ORBIT_CNT').max().hist('BX_COUNTER',bins=200)

In [ ]:
df.groupby('ORBIT_CNT').max()

In [ ]:
num_orb=df[df['BX_COUNTER']==3563].shape[0]
orb_size=df.groupby('ORBIT_CNT').max().loc[:,"BX_COUNTER"].max()

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [ ]:
max_orb=df["ORBIT_CNT"].max()
min_orb=df["ORBIT_CNT"].min()
n_orb=max_orb-min_orb
n_orb
#every orbit lasts 3563 BX_COUNTERs, except for the last one
#calculate time in ns

bx_lorb=df[df["ORBIT_CNT"]==max_orb].loc[:,"BX_COUNTER"].max()
tm_lorb=df[df["ORBIT_CNT"]==max_orb][df["BX_COUNTER"]==bx_lorb].loc[:,"TDC_MEAS"].max()
tot_time=(n_orb-1)*orb_size*25+(bx_lorb-1)*25+tm_lorb*(25/30)
print("Total time: ", tot_time)

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
df['ABS_TIME']=pd.Series((df['ORBIT_CNT']-min_orb)*orb_size*25+df['BX_COUNTER']*25+df['TDC_MEAS']*(25/30))
df

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [ ]:
df['HEAD']=pd.Series(np.random.randint(0,2,df.shape[0]))
df

6\. Create a new DataFrame that contains only the rows with HEAD=1.

In [ ]:
df_new=df[df['HEAD']==1]
df_new

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
plt.hist(df[df['FPGA']==0]['TDC_CHANNEL'])

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

In [ ]:
df.groupby('TDC_CHANNEL').size().sort_values(ascending=False).head(3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
df.groupby('ORBIT_CNT').size().sort_values(ascending=True)